<a href="https://colab.research.google.com/github/lyuyanyii/text-classification/blob/main/newsgroup_trained_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os

os.system('wget https://github.com/lil-lab/lm-class/raw/refs/heads/main/assignments/a1/starter-repo/data/newsgroups/train/train_data.csv')


0

In [2]:
os.system('wget https://github.com/lil-lab/lm-class/raw/refs/heads/main/assignments/a1/starter-repo/data/newsgroups/train/train_labels.csv')


0

In [33]:
import csv
import string

D = 128

def data_process(raw_text : string):
    text = raw_text.lower()
    raw_word_list = text.split()
    word_list = []
    for raw_word in raw_word_list:
        word = ''.join([i for i in raw_word if i.isalpha()])
        if word != '':
            word_list.append(word)
    return word_list

"""
def bag_of_words_embedding(word_vector_dict : dict, word_list : list):
    vector_list = []

    for word in word_list:
        if word not in word_vector_dict:
            word_vector_dict[word] = np.random.normal(loc = 0, scale = 0.5, size = (D, 1))

        embedding = word_vector_dict[word]
        vector_list.append(embedding)

    bag_of_words = np.concatenate(vector_list, axis = 1).mean(axis = 1)
    return bag_of_words
"""

train_data = []
# word_vector_dict = {}

with open('train_data.csv', newline = '') as file:
    csv_file = csv.DictReader(file)
    for line in csv_file:
        word_list = data_process(line['text'])

        train_data.append(word_list)

print(len(train_data))

9051


In [66]:
class Word_Embedding:
    def __init__(self, dataset):
        self.embedding = {}

        for text in dataset:
            for word in text:
                if word not in self.embedding:
                    self.embedding[word] = np.random.normal(loc = 0, scale = len(text)**0.5, size = (D, 1))

    def forward(self, text_batch):
        self.x = []
        self.text_batch = text_batch
        for text in text_batch:
            bag_of_words = []
            for word in text:
                bag_of_words.append( self.embedding[word] )
            bag_of_words = np.concatenate( bag_of_words, axis = 1).mean(axis = 1)
            bag_of_words = bag_of_words.reshape((1, bag_of_words.shape[0], 1))
            self.x.append( bag_of_words )
        self.x = np.concatenate(self.x, axis = 0)
        return self.x

    def back_prop(self, grad_x):
        self.grad_embedding = {}

        for i in range(len(text_batch)):
            text = text_batch[i]

            for word in text:
                if word not in self.grad_embedding:
                    self.grad_embedding[word] = np.zeros((D, 1))
                self.grad_embedding[word] += grad_x[i] / len(text) / len(text_batch)

    def update(self, lr):
        for (word, grad) in self.grad_embedding.items():
            self.embedding[word] += grad * lr


In [35]:
train_labels = []
label_index_map = {}

num_class = 0

with open('train_labels.csv', newline = '') as file:
    csv_file = csv.DictReader(file)
    for line in csv_file:
        label = line['newsgroup']

        if label not in label_index_map:
            label_index_map[label] = num_class
            num_class += 1

        index = label_index_map[label]
        train_labels.append(index)

print(num_class)
print(len(train_labels))

20
9051


In [36]:
train_ratio = 0.8
dataset_size = len(train_data)

valid_data = train_data[ int(dataset_size * train_ratio) : ]
valid_labels = train_labels[ int(dataset_size * train_ratio) : ]

train_data = train_data[ : int(dataset_size * train_ratio) ]
train_labels = train_labels[ : int(dataset_size * train_ratio) ]

print(len(train_data))
print(len(valid_data))

7240
1811


In [46]:
class Linear:
    def __init__(self, in_dim, out_dim):
        self.w = np.random.normal(loc = 0, scale = 1 / in_dim**0.5, size = (out_dim, in_dim))
        self.b = np.random.normal(loc = 0, scale = 1 / in_dim**0.5, size = (out_dim, 1))
        self.grad_w = np.zeros((out_dim, in_dim))
        self.grad_b = np.zeros((out_dim, 1))

    def forward(self, x):
        self.x = x
        self.out = np.matmul(self.w, x) + self.b
        return self.out

    def back_prop(self, grad_out):
        self.grad_b = grad_out.mean(axis = 0)
        self.grad_w = np.matmul(grad_out, self.x.transpose(0, 2, 1)).mean(axis = 0)
        self.grad_x = np.matmul(self.w.transpose((1, 0)), grad_out)
        return self.grad_x

    def update(self, lr):
        self.w += self.grad_w * lr
        self.b += self.grad_b * lr


In [47]:
class Relu:
    def __init__(self):
        pass

    def forward(self, x):
        self.mask = (x >= 0)
        return x * self.mask

    def back_prop(self, grad_y):
        return grad_y * self.mask


In [48]:
class Softmax:
    def __init__(self):
        pass

    def forward(self, x):
        self.x = x
        exp_x = np.exp(x)
        d1, d2, d3 = x.shape
        self.y = exp_x / exp_x.sum(axis = 1).reshape(d1, 1, 1)
        return self.y

    def back_prop(self, grad_y):
        (d1, d2, d3) = grad_y.shape
        # print((np.identity(d2) - self.y).shape)
        # print((self.y.transpose(0, 2, 1)* (np.identity(d2, ) - self.y)).shape)
        # print(self.y.transpose(0, 2, 1))
        # print(self.y.transpose(0, 2, 1)* (np.identity(d2, ) - self.y))
        grad_x = np.matmul( (self.y.transpose(0, 2, 1) * (np.identity(d2) - self.y)), grad_y )
        return grad_x

"""
x = np.arange(0, 3)
x = x.reshape((1, 3, 1))
sft_layer = Softmax()
y = sft_layer.forward(x)
print(y)
grad_y = np.array([1,0,0])
grad_y = grad_y.reshape((1, 3, 1))
grad_x = sft_layer.back_prop(grad_y)
print(grad_x)
"""

'\nx = np.arange(0, 3)\nx = x.reshape((1, 3, 1))\nsft_layer = Softmax()\ny = sft_layer.forward(x)\nprint(y)\ngrad_y = np.array([1,0,0])\ngrad_y = grad_y.reshape((1, 3, 1))\ngrad_x = sft_layer.back_prop(grad_y)\nprint(grad_x)\n'

In [ ]:
batch_size = 10
depth = 3

embedding_layer = Word_Embedding(train_data + valid_data)
network = []
for i in range(depth - 1):
    network.append((Linear(D, D), Relu()))
network.append((Linear(D, num_class), Softmax()))

def train(embedding_layer, network, data, labels, lr):
    x = embedding_layer.forward(data)
    for (layer, f_layer) in network:
        x = f_layer.forward(layer.forward(x))

    # loss1 = - np.log(np.vstack([x[i][labels[i]][0] for i in range(d1)]))
    d1, d2, d3 = x.shape
    loss = - np.log(x[range(d1), labels, 0]).mean()
    grad_x = np.zeros((d1, d2, d3))
    for i in range(d1):
        grad_x[i, labels[i], 0] = 1/x[i, labels[i], 0]

    for (layer, f_layer) in reversed(network):
        grad_y1 = f_layer.back_prop(grad_x)
        grad_y2 = layer.back_prop(grad_y1)
        grad_x = grad_y2
        layer.update(lr)

    embedding_layer.back_prop(grad_x)
    embedding_layer.update(lr)

    x = x.reshape(d1, d2)
    idx = np.argmax(x, axis = 1)
    acc = np.array(idx == labels).mean()

    return loss, acc

def eval(embedding_layer, network, data):
    x = embedding_layer.forward(data)
    for (layer, f_layer) in network:
        x = f_layer.forward(layer.forward(x))

    d1, d2, d3 = x.shape
    x = x.reshape(d1, d2)
    idx = np.argmax(x, axis = 1)
    return idx

num_epochs = 100

for i in range(num_epochs):
    loss = 0

    idx = list(np.random.permutation(len(train_data)))
    train_data = [train_data[i] for i in idx]
    train_labels = [train_labels[i] for i in idx]

    print('---- Epoch ', i, '----')
    acc_list = []
    loss_list = []
    for j in range(len(train_data) // batch_size):
        start = j * batch_size
        end = min(start + batch_size, len(train_data))
        loss, acc = train(embedding_layer, network, train_data[start:end], train_labels[start:end], lr = 0.1 * (num_epochs - i) / num_epochs + 0.01)
        acc_list.append(acc)
        loss_list.append(loss)
    print('Loss =', np.array(loss_list).mean(), ', Training Acc =', np.array(acc_list).mean())

    pred = eval(embedding_layer, network, valid_data)
    acc = np.array(pred == valid_labels).mean()
    print('Validation Accurary = ', acc)


---- Epoch  0 ----
Loss = 2.9666647732208076 , Training Acc = 0.08038674033149171
Validation Accurary =  0.09939260077305356
---- Epoch  1 ----
Loss = 2.806166190511195 , Training Acc = 0.12582872928176794
Validation Accurary =  0.13694091662065158
---- Epoch  2 ----
Loss = 2.6469282555539837 , Training Acc = 0.1743093922651934
Validation Accurary =  0.18553285477636663
---- Epoch  3 ----
Loss = 2.5192083602466586 , Training Acc = 0.22168508287292818
Validation Accurary =  0.22473771397018222
---- Epoch  4 ----
Loss = 2.4050731464862296 , Training Acc = 0.26160220994475136
Validation Accurary =  0.26339039204859194
---- Epoch  5 ----
Loss = 2.3229607754373385 , Training Acc = 0.28604972375690607
Validation Accurary =  0.27277747101049143
---- Epoch  6 ----
Loss = 2.235444875434165 , Training Acc = 0.313121546961326
Validation Accurary =  0.29044726670347876
---- Epoch  7 ----
Loss = 2.178817147145349 , Training Acc = 0.33259668508287293
Validation Accurary =  0.2887907233572612
---- Ep